In [401]:
import itertools
import time

import numpy as np
import pandas as pd
import tensorflow as tf

In [254]:
INPUT_ROOT = '/Users/hkj/Projects/ml/datasets/ml-latest-small'

USER_NUM = 672+1
ITEM_NUM = 9125
DIMS = 15
LEARNING_RATE = 0.001
REG = 0.05
BATCH_SIZE = 1000
EPOCHS = 100

In [234]:
def transform_ratings():
    movies = {}
    movies_fp = open(INPUT_ROOT + '/movies.csv')
    lines = movies_fp.readlines()
    movies_fp.close()
    for i, line in enumerate(lines[1:]):
        delim = line.find(',')
        mov_id = line[:delim]
        movies[mov_id] = i
        
    transform_fp = open(INPUT_ROOT + '/transformed_movies.csv', 'w')
    for k, v in movies.items():
        transform_fp.write('{0},{1}\n'.format(v, k))
    transform_fp.close()

    ratings_fp = open(INPUT_ROOT + '/ratings.csv')
    lines = ratings_fp.readlines()
    ratings_fp.close()
    
    transform_fp = open(INPUT_ROOT + '/transformed_ratings.csv', 'w')
    for line in lines:
        parts = line.strip().split(',')
        mov_id = parts[1]
        parts[1] = str(movies.get(mov_id, mov_id))
        new_line = ','.join(parts)
        transform_fp.write(new_line + '\n')
    transform_fp.close()
    
#transform_ratings()

In [267]:
def load_movie_lens():
    df = pd.read_csv(INPUT_ROOT + '/transformed_ratings.csv')
    mask = np.random.rand(len(df)) < 0.9
    return df[mask].reset_index(drop=True), df[~mask].reset_index(drop=True)

def get_next_batch(df, batch_size=BATCH_SIZE):
    idx = np.random.choice(len(df), size=batch_size)
    batch = df.iloc[idx].reset_index(drop=True)
    return batch['userId'], batch['movieId'], batch['rating']

def clip(x):
    return np.clip(x, 1.0, 5.0)

In [364]:
from sklearn.base import BaseEstimator

class SVDRecommender(BaseEstimator):
    
    def __init__(self, user_num, item_num, batch_size=BATCH_SIZE, dims=DIMS, 
                 learning_rate=LEARNING_RATE, reg=REG):
        self.user_num = user_num
        self.item_num = item_num
        self.batch_size = batch_size
        self.dims = dims
        self.learning_rate = learning_rate
        self.reg = reg
        self._session = None
        
    def _build_graph(self):
        user_batch = tf.placeholder(tf.int32, shape=[None], name='user_batch')
        item_batch = tf.placeholder(tf.int32, shape=[None], name='item_batch')
        rating_batch = tf.placeholder(tf.float32, shape=[None], name='rating_batch')
        
        with tf.name_scope('svd'):
            bias_global = tf.get_variable("bias_global", shape=[])
            w_bias_user = tf.get_variable("embd_bias_user", shape=[self.user_num])
            w_bias_item = tf.get_variable("embd_bias_item", shape=[self.item_num])
            bias_user = tf.nn.embedding_lookup(w_bias_user, user_batch, name="bias_user")
            bias_item = tf.nn.embedding_lookup(w_bias_item, item_batch, name="bias_item")
            w_user = tf.get_variable("embd_user", shape=[self.user_num, self.dims], 
                                     initializer=tf.truncated_normal_initializer(stddev=0.02))
            w_item = tf.get_variable("embd_item", shape=[self.item_num, self.dims],
                                     initializer=tf.truncated_normal_initializer(stddev=0.02))
            embd_user = tf.nn.embedding_lookup(w_user, user_batch, name="embedding_user")
            embd_item = tf.nn.embedding_lookup(w_item, item_batch, name="embedding_item")

        with tf.name_scope('predict'):
            user_item_prod = tf.reduce_sum(tf.multiply(embd_user, embd_item), 1)
            pred = tf.add(tf.add(tf.add(user_item_prod, bias_global), bias_user), bias_item, name='pred')
            regularizer = tf.add(tf.nn.l2_loss(embd_user), tf.nn.l2_loss(embd_item), name="svd_regularizer")

        with tf.name_scope('loss'):
            cost_l2 = tf.nn.l2_loss(tf.subtract(pred, rating_batch))
            penalty = tf.constant(self.reg, dtype=tf.float32, shape=[], name="l2")
            cost = tf.add(cost_l2, tf.multiply(regularizer, penalty))
            global_step = tf.train.get_global_step()
            optimizer = tf.train.AdamOptimizer(learning_rate=self.learning_rate)
            training_op = optimizer.minimize(cost, global_step=global_step)
            
        init = tf.global_variables_initializer()
        saver = tf.train.Saver()
            
        self._user_batch = user_batch
        self._item_batch = item_batch
        self._rating_batch = rating_batch
        self._cost = cost
        self._pred, self._training_op = pred, training_op
        self._init, self._saver = init, saver
        
    def close_session(self):
        if self._session:
            self._session.close()
            
    def _get_next_batch(self, X_users, X_items, y):
        idx = np.random.choice(len(X_users), size=self.batch_size)
        return X_users[idx], X_items[idx], y[idx]
    
    def _get_model_params(self):
        """Get all variable values (used for early stopping, faster than saving to disk)"""
        with self._graph.as_default():
            gvars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
        return {gvar.op.name: value for gvar, value in zip(gvars, self._session.run(gvars))}
    
    def _restore_model_params(self, model_params):
        """Set all variables to the given values (for early stopping, faster than loading from disk)"""
        gvar_names = list(model_params.keys())
        assign_ops = {gvar_name: self._graph.get_operation_by_name(gvar_name + "/Assign")
                      for gvar_name in gvar_names}
        init_values = {gvar_name: assign_op.inputs[1] for gvar_name, assign_op in assign_ops.items()}
        feed_dict = {init_values[gvar_name]: model_params[gvar_name] for gvar_name in gvar_names}
        self._session.run(assign_ops, feed_dict=feed_dict)
           
    def fit(self, X, y, n_epochs=100, X_valid=None, y_valid=None):
        self.close_session()
        
        X_users = X[:,0]
        X_items = X[:,1]
        self._graph = tf.Graph()
        with self._graph.as_default():
            self._build_graph()
            
        best_loss = np.infty
        best_params = None
        epochs_without_improvement = 0
        
        self._session = tf.Session(graph=self._graph)
        start = time.time()
        with self._session.as_default() as sess:
            self._init.run()
            for epoch in range(n_epochs):
                for batch in range(len(X) // self.batch_size):
                    users, items, ratings = self._get_next_batch(X_users, X_items, y)
                    pred_train, _ = sess.run([self._pred, self._training_op], feed_dict={
                        self._user_batch: X_users, 
                        self._item_batch: X_items, 
                        self._rating_batch: y})
                pred_train = clip(pred_train)
                squared_train_err = np.power(pred_train - y, 2)
                rmse_train = np.sqrt(np.mean(squared_train_err))
                
                if X_valid is not None and y_valid is not None:
                    X_valid_users = X_valid[:,0]
                    X_valid_items = X_valid[:,1]
                    pred_test = sess.run(self._pred, feed_dict={
                        self._user_batch: X_valid_users, 
                        self._item_batch: X_valid_items, 
                        self._rating_batch: y_valid,
                    })
                    pred_test = clip(pred_test)
                    squared_test_err = np.power(pred_test - y_valid, 2)
                    rmse_test = np.sqrt(np.mean(squared_test_err))
                    print("{}\tValidation error: {:.6f}\tTrain error: {:.6f}\tTime elapsed: {:.2f}s".format(
                        epoch, rmse_test, rmse_train, time.time() - start))
                    if rmse_test < best_loss:
                        best_loss = rmse_test
                        best_params = self._get_model_params()
                        epochs_without_improvement = 0
                    else:
                        epochs_without_improvement += 1
                    if epochs_without_improvement == 20:
                        print('Early stopping')
                        break
                else:
                    print("{}\tTrain error: {:.6f}\tTime elapsed: {:.2f}".format(
                        epoch, rmse_train, time.time() - start))
            if best_params:
                self._restore_model_params(best_params)
            return self
        
    def predict(self, X):
        if not self._session:
            raise NotFittedError("This %s instance is not fitted yet" % self.__class__.__name__)
        X_pred_users = X[:,0]
        X_pred_items = X[:,1]
        with self._session.as_default() as sess:
            results = self._pred.eval(feed_dict={
                self._user_batch: X_pred_users, 
                self._item_batch: X_pred_items, 
            })
            return clip(results)
        
    def save(self, path):
        self._saver.save(self._session, path)

In [362]:
np.random.seed(42)
train, test = load_movie_lens()
print(train.shape, test.shape)
print('Total ratings:', train.shape[0] + test.shape[0])

X_train = np.array(train.iloc[:,0:2])
y_train = np.array(train.iloc[:,2])
X_test = np.array(test.iloc[:,0:2])
y_test = np.array(test.iloc[:,2])

estimator = SVDRecommender(USER_NUM, ITEM_NUM)
estimator.fit(X_train, y_train, X_valid=X_test, y_valid=y_test)
estimator.save('/Users/hkj/Projects/ml/cinefire_02_17.ckpt')

(90220, 4) (9830, 4)
Total ratings: 100050
0	Validation error: 2.350136	Train error: 2.345306	Time elapsed: 2.51s
1	Validation error: 1.592279	Train error: 1.561560	Time elapsed: 4.91s
2	Validation error: 1.083035	Train error: 0.993001	Time elapsed: 7.25s
3	Validation error: 0.975352	Train error: 0.846110	Time elapsed: 9.57s
4	Validation error: 0.963602	Train error: 0.820902	Time elapsed: 11.86s
5	Validation error: 0.961979	Train error: 0.814020	Time elapsed: 14.21s
6	Validation error: 0.961315	Train error: 0.810340	Time elapsed: 16.51s
7	Validation error: 0.960689	Train error: 0.807102	Time elapsed: 18.84s
8	Validation error: 0.959886	Train error: 0.803408	Time elapsed: 21.20s
9	Validation error: 0.958930	Train error: 0.798726	Time elapsed: 23.56s
10	Validation error: 0.957831	Train error: 0.792668	Time elapsed: 25.87s
11	Validation error: 0.956636	Train error: 0.784976	Time elapsed: 28.17s
12	Validation error: 0.955398	Train error: 0.775508	Time elapsed: 30.50s
13	Validation error: 0

In [318]:
def load_movie_transform_index():
    fp = open(INPUT_ROOT + '/transformed_movies.csv')
    lines = fp.readlines()
    fp.close()
    index = {}
    for line in lines:
        parts = line.strip().split(',')
        index[int(parts[0])] = int(parts[1])
    return index

def load_movie_master():
    movies = {}
    movies_fp = open(INPUT_ROOT + '/movies.csv')
    lines = movies_fp.readlines()
    movies_fp.close()
    for i, line in enumerate(lines):
        if i == 0:
            continue
        delim = line.find(',')
        mov_id = line[:delim]
        movies[int(mov_id)] = line
    return movies

MOVIE_TRANSFORM_INDEX = load_movie_transform_index()
MOVIE_MASTER = load_movie_master()

In [319]:
def lookup_movie(movie_id):
    old_id = MOVIE_TRANSFORM_INDEX[movie_id]
    return MOVIE_MASTER[old_id]

In [343]:
tf.reset_default_graph()

class Prediction(object):

    def __init__(self, idx, predictions):
        self.idx = idx
        self.old_idx = MOVIE_TRANSFORM_INDEX[idx]
        self.rating = predictions[idx]
        self.movie_info = lookup_movie(idx).strip()
    
def get_recommendations(uid, k=100):
    movies = [m for m in range(ITEM_NUM)]
    X_pred = []
    for m in range(ITEM_NUM):
        X_pred.append([uid, m])
    predictions = estimator.predict(np.array(X_pred))
    top_k = predictions.argsort()[-k:][::-1]
    result = []
    for i in top_k:
        result.append(Prediction(i, predictions))
    return result

def get_explicit_ratings(uid):
    ratings_fp = open(INPUT_ROOT + '/transformed_ratings.csv')
    lines = ratings_fp.readlines()
    results = []
    for line in lines:
        if line.startswith('{0},'.format(uid)):            
            parts = line.strip().split(',')
            movie = int(parts[1])
            results.append((movie, lookup_movie(movie).strip(), parts[2]))
    return results

In [369]:
UID = 672
explicit = get_explicit_ratings(UID)
recs = get_recommendations(UID, k=10)

rated_movies = {}

print('User Rated Movies')
print('==================')
for r in explicit:
    rated_movies[r[0]] = r[2]
    print(r[1], r[2])
    
print('\nRecommended Movies')
print('====================')
for r in recs:
    if r.idx in rated_movies:
        print('[USER RATED {0}]'.format(rated_movies[r.idx]), r.rating, r.movie_info)
    else:
        print('     [RECOMMENDED {0},{1}]'.format(r.old_idx, r.idx), r.rating, r.movie_info)

User Rated Movies
89745,"Avengers, The (2012)",Action|Adventure|Sci-Fi|IMAX 5.0
85438,Jane Eyre (2011),Drama|Romance 1.0
6306,I Am Trying to Break Your Heart (2002),Documentary 1.0
122892,Avengers: Age of Ultron (2015),Action|Adventure|Sci-Fi 4.5
4732,Bubble Boy (2001),Comedy 1.5
4731,Innocence (2000),Drama 1.0
59315,Iron Man (2008),Action|Adventure|Sci-Fi 5.0
91690,Friends with Kids (2011),Comedy 1.0
59684,Lake of Fire (2006),Documentary 1.0
77837,You Don't Know Jack (2010),Drama 1.5
67504,Land of Silence and Darkness (Land des Schweigens und der Dunkelheit) (1971),Documentary 1.0
5427,Caveman (1981),Comedy 1.0
83318,"Goat, The (1921)",Comedy 1.0
110102,Captain America: The Winter Soldier (2014),Action|Adventure|Sci-Fi|IMAX 5.0
88140,Captain America: The First Avenger (2011),Action|Adventure|Sci-Fi|Thriller|War 4.5
122920,Captain America: Civil War (2016),Action|Sci-Fi|Thriller 5.0
93721,Jiro Dreams of Sushi (2011),Documentary 1.0
62718,"Angus, Thongs and Perfect Snogging (2008)",Come

In [453]:
def custom_svd_model(features, labels, mode, params):
    user_batch = features['userId']
    item_batch = features['movieId']
    user_num = params['n_users']
    item_num = params['n_items']
    dims = params.get('dims', DIMS)
    learning_rate = params.get('learning_rate', LEARNING_RATE)
    reg = params.get('reg', REG)

    with tf.name_scope('svd'):
        bias_global = tf.get_variable("bias_global", shape=[])
        w_bias_user = tf.get_variable("embd_bias_user", shape=[user_num])
        w_bias_item = tf.get_variable("embd_bias_item", shape=[item_num])
        bias_user = tf.nn.embedding_lookup(w_bias_user, user_batch, name="bias_user")
        bias_item = tf.nn.embedding_lookup(w_bias_item, item_batch, name="bias_item")
        w_user = tf.get_variable("embd_user", shape=[user_num, dims], 
                                 initializer=tf.truncated_normal_initializer(stddev=0.02))
        w_item = tf.get_variable("embd_item", shape=[item_num, dims],
                                 initializer=tf.truncated_normal_initializer(stddev=0.02))
        embd_user = tf.nn.embedding_lookup(w_user, user_batch, name="embedding_user")
        embd_item = tf.nn.embedding_lookup(w_item, item_batch, name="embedding_item")

    with tf.name_scope('predict'):
        user_item_prod = tf.reduce_sum(tf.multiply(embd_user, embd_item), 1)
        pred = tf.add(tf.add(tf.add(user_item_prod, bias_global), bias_user), bias_item, name='pred')
        regularizer = tf.add(tf.nn.l2_loss(embd_user), tf.nn.l2_loss(embd_item), name="svd_regularizer")

    export_outputs = {
        'pred': tf.estimator.export.PredictOutput(outputs={'pred': pred})
    }
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode, predictions=pred, export_outputs=export_outputs)

    with tf.name_scope('svd_loss'):
        cost_l2 = tf.nn.l2_loss(tf.subtract(pred, labels))
        penalty = tf.constant(reg, dtype=tf.float32, shape=[], name="l2")
        cost = tf.add(cost_l2, tf.multiply(regularizer, penalty), name='loss')
        global_step = tf.train.get_global_step()
        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
        training_op = optimizer.minimize(cost, global_step=global_step)
         
    return tf.estimator.EstimatorSpec(mode, loss=cost, train_op=training_op, export_outputs=export_outputs)
    
def custom_input_fn(X, y, batch_size):
    idx = np.random.choice(len(X), size=batch_size)
    X_users = X[:,0]
    X_items = X[:,1]
    features = {
        'userId': tf.constant(X_users[idx], dtype=tf.int32),
        'movieId': tf.constant(X_items[idx], dtype=tf.int32)
    }
    return features, tf.constant(y[idx], dtype=tf.float32)
    
my_feature_columns = []
for key in ['userId', 'movieId']:
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))
    
classifier = tf.estimator.Estimator(
    model_fn=custom_svd_model,
    params={
        'feature_columns': my_feature_columns,
        'n_users': USER_NUM,
        'n_items': ITEM_NUM,
    })

np.random.seed(42)
train, test = load_movie_lens()
print(train.shape, test.shape)
print('Total ratings:', train.shape[0] + test.shape[0])

X_train = np.array(train.iloc[:,0:2])
y_train = np.array(train.iloc[:,2])
X_test = np.array(test.iloc[:,0:2])
y_test = np.array(test.iloc[:,2])

# Train the Model.
classifier.train(
    input_fn=lambda:custom_input_fn(X_train, y_train, BATCH_SIZE), steps=50000)

INFO:tensorflow:Using default config.


INFO:tensorflow:Using default config.


INFO:tensorflow:Using config: {'_model_dir': '/var/folders/77/6t23tk2x0wjcb919ypy3wqhm0081qz/T/tmp73xojtqn', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x12f511d30>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': '/var/folders/77/6t23tk2x0wjcb919ypy3wqhm0081qz/T/tmp73xojtqn', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x12f511d30>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


(90220, 4) (9830, 4)
Total ratings: 100050
INFO:tensorflow:Create CheckpointSaverHook.


...

INFO:tensorflow:loss = 1.1366103, step = 49901 (0.136 sec)


INFO:tensorflow:Saving checkpoints for 50000 into /var/folders/77/6t23tk2x0wjcb919ypy3wqhm0081qz/T/tmp73xojtqn/model.ckpt.


INFO:tensorflow:Saving checkpoints for 50000 into /var/folders/77/6t23tk2x0wjcb919ypy3wqhm0081qz/T/tmp73xojtqn/model.ckpt.


INFO:tensorflow:Loss for final step: 1.1366118.


INFO:tensorflow:Loss for final step: 1.1366118.


In [450]:
def get_recommendations_2(uid, k=100):
    movies = [m for m in range(ITEM_NUM)]
    def pred_input_fn():        
        features = {
            'userId': tf.constant([uid], dtype=tf.int32),
            'movieId': tf.constant(movies, dtype=tf.int32)
        }
        return features, None
    cps = classifier.predict(input_fn=pred_input_fn)
    predictions = np.array(list(itertools.islice(cps, len(movies))))
    top_k = predictions.argsort()[-k:][::-1]
    result = []
    predictions = clip(predictions)
    for i in top_k:
        result.append(Prediction(i, predictions))
    return result

In [451]:
UID = 672
explicit = get_explicit_ratings(UID)
recs = get_recommendations_2(UID, k=100)

rated_movies = {}

print('User Rated Movies')
print('==================')
for r in explicit:
    rated_movies[r[0]] = r[2]
    print(r[1], r[2])
    
print('\nRecommended Movies')
print('====================')
for r in recs:
    if r.idx in rated_movies:
        print('[USER RATED {0}]'.format(rated_movies[r.idx]), r.rating, r.movie_info)
    else:
        print('     [RECOMMENDED {0},{1}]'.format(r.old_idx, r.idx), r.rating, r.movie_info)

INFO:tensorflow:Restoring parameters from /var/folders/77/6t23tk2x0wjcb919ypy3wqhm0081qz/T/tmpwuxn727g/model.ckpt-50000


INFO:tensorflow:Restoring parameters from /var/folders/77/6t23tk2x0wjcb919ypy3wqhm0081qz/T/tmpwuxn727g/model.ckpt-50000


User Rated Movies
89745,"Avengers, The (2012)",Action|Adventure|Sci-Fi|IMAX 5.0
85438,Jane Eyre (2011),Drama|Romance 1.0
6306,I Am Trying to Break Your Heart (2002),Documentary 1.0
122892,Avengers: Age of Ultron (2015),Action|Adventure|Sci-Fi 4.5
4732,Bubble Boy (2001),Comedy 1.5
4731,Innocence (2000),Drama 1.0
59315,Iron Man (2008),Action|Adventure|Sci-Fi 5.0
91690,Friends with Kids (2011),Comedy 1.0
59684,Lake of Fire (2006),Documentary 1.0
77837,You Don't Know Jack (2010),Drama 1.5
67504,Land of Silence and Darkness (Land des Schweigens und der Dunkelheit) (1971),Documentary 1.0
5427,Caveman (1981),Comedy 1.0
83318,"Goat, The (1921)",Comedy 1.0
110102,Captain America: The Winter Soldier (2014),Action|Adventure|Sci-Fi|IMAX 5.0
88140,Captain America: The First Avenger (2011),Action|Adventure|Sci-Fi|Thriller|War 4.5
122920,Captain America: Civil War (2016),Action|Sci-Fi|Thriller 5.0
93721,Jiro Dreams of Sushi (2011),Documentary 1.0
62718,"Angus, Thongs and Perfect Snogging (2008)",Come

In [454]:
def serving_input_fn():
    inputs = {
        'userId': tf.placeholder(dtype=tf.int32, shape=(1,None)),
        'movieId': tf.placeholder(dtype=tf.int32, shape=(1,None)),
    }
    return tf.estimator.export.ServingInputReceiver(inputs, inputs)

classifier.export_savedmodel('/Users/hkj/Projects/ml/cinefire_tf', 
                             serving_input_receiver_fn=serving_input_fn)

INFO:tensorflow:Restoring parameters from /var/folders/77/6t23tk2x0wjcb919ypy3wqhm0081qz/T/tmp73xojtqn/model.ckpt-50000


INFO:tensorflow:Restoring parameters from /var/folders/77/6t23tk2x0wjcb919ypy3wqhm0081qz/T/tmp73xojtqn/model.ckpt-50000


INFO:tensorflow:Assets added to graph.


INFO:tensorflow:Assets added to graph.


INFO:tensorflow:No assets to write.


INFO:tensorflow:No assets to write.


INFO:tensorflow:SavedModel written to: b"/Users/hkj/Projects/ml/cinefire_tf/temp-b'1518912955'/saved_model.pb"


INFO:tensorflow:SavedModel written to: b"/Users/hkj/Projects/ml/cinefire_tf/temp-b'1518912955'/saved_model.pb"


b'/Users/hkj/Projects/ml/cinefire_tf/1518912955'

In [474]:
def train_sklearn_estimator_and_export(path):
    np.random.seed(42)

    train, test = load_movie_lens()
    X_train = np.array(train.iloc[:,0:2])
    y_train = np.array(train.iloc[:,2])
    X_test = np.array(test.iloc[:,0:2])
    y_test = np.array(test.iloc[:,2])

    estimator = SVDRecommender(USER_NUM, ITEM_NUM)
    estimator.fit(X_train, y_train, X_valid=X_test, y_valid=y_test)
    
    with estimator._graph.as_default():
        with estimator._session.as_default() as sess:
            builder = tf.saved_model.builder.SavedModelBuilder(export_path)
            # Build the signature_def_map.
            prediction_signature = (
                  tf.saved_model.signature_def_utils.build_signature_def(
                      inputs = {
                          'userId': tf.saved_model.utils.build_tensor_info(estimator._user_batch),
                          'movieId': tf.saved_model.utils.build_tensor_info(estimator._item_batch),
                      },
                      outputs={'scores': tf.saved_model.utils.build_tensor_info(estimator._pred)},
                      method_name=tf.saved_model.signature_constants.PREDICT_METHOD_NAME))

            builder.add_meta_graph_and_variables(
              sess, [tf.saved_model.tag_constants.SERVING],
              signature_def_map={
                  'predict_ratings': prediction_signature,
              })

            builder.save()
            
#export_path = '/Users/hkj/Projects/ml/cinefire_tf/001'
#train_sklearn_estimator_and_export(export_path)

0	Validation error: 2.757787	Train error: 2.754556	Time elapsed: 2.58s
1	Validation error: 2.757787	Train error: 2.754556	Time elapsed: 5.24s
2	Validation error: 2.300183	Train error: 2.288921	Time elapsed: 7.63s
3	Validation error: 1.391138	Train error: 1.315757	Time elapsed: 9.93s
4	Validation error: 1.058020	Train error: 0.919723	Time elapsed: 12.22s
5	Validation error: 0.998390	Train error: 0.831329	Time elapsed: 14.52s
6	Validation error: 0.990245	Train error: 0.811144	Time elapsed: 16.85s
7	Validation error: 0.989218	Train error: 0.803822	Time elapsed: 19.16s
8	Validation error: 0.989102	Train error: 0.799552	Time elapsed: 21.51s
9	Validation error: 0.989065	Train error: 0.796167	Time elapsed: 23.86s
10	Validation error: 0.989078	Train error: 0.793022	Time elapsed: 26.35s
11	Validation error: 0.989151	Train error: 0.789890	Time elapsed: 28.69s
12	Validation error: 0.989303	Train error: 0.786690	Time elapsed: 31.01s
13	Validation error: 0.989544	Train error: 0.783409	Time elapsed:

INFO:tensorflow:No assets to save.


INFO:tensorflow:No assets to write.


INFO:tensorflow:No assets to write.


INFO:tensorflow:SavedModel written to: b'/Users/hkj/Projects/ml/cinefire_tf/001/saved_model.pb'


INFO:tensorflow:SavedModel written to: b'/Users/hkj/Projects/ml/cinefire_tf/001/saved_model.pb'
